#  *The best locations where it pays to open restaurant in New York*

## Introduction/Business Problem

Restaurants are one of the most profitable sectors. However, according to one study, 60 percent close or change owners within the first year of operation, 80 percent fail within five years. Usually, restaurants fail with combination of problems that eventually lead to their closure. A bad location is one of the biggest reasons for restaurant failure. For example, a restaurant can sell the best "burger" in the world. If it is in a poor location (hidden, sparsely inhabited, blind and difficult to access) it will have to put in much more effort to fetch customers than to serve them.

In this context, how to define the best locations where it pays to open a restaurant?

Our objective is to recommend the best locations in New York city (well inhabited, close to subways, distant from existing restaurants) to open restaurant. We don’t distinguish the kind of restaurant.



## Data

Location data and demographic data of neighborhoods from https://data.cityofnewyork.us. 
The link of the CSV file is https://query.data.world/s/ouharsff2db3gnsqgckj65osbyog4y



Location data of New York city subway station from https://data.cityofnewyork.us. 
The link of the CSV file is https://query.data.world/s/rttrjnk7raatdri6ialljpsucvbv5b
It will help to determine the minimal distance from a neighborhood location to a subway station and the number of subways located in a given radius.


Location data of restaurants provided from Foursquare API. It will help to determine the minimal distance from a restaurant to a neighborhood location and the number of restaurants located in a given radius.

So, we generate a table with a number of rows corresponding to neighborhoods and columns are:
- longitude and latitude
- population
- minimal distance from a neighborhood location to a subway station
- number of subways located in a given radius
- minimal distance from a restaurant to a neighborhood location
- number of restaurants located in a given radius


The best locations are those where there is no or few restaurants, close to subway stations and well inhabited




